In [ ]:
# CELL 1.1 : accessing to data 
# Needs [ CELL 0 : import ]

# defines variables 

#-----------------------------------------------------------------------------

path_training = 'Retrieving_data\BelgiumTSC_Training\Training'
path_testing = 'Retrieving_data\BelgiumTSC_Testing\Testing'

nbr_class = 62 # len(next(walk(path_training))[1])  counting the number of classes 
print('number of classes : ', nbr_class)

#-----------------------------------------------------------------------------

# resol is the format for the images that we want for convenience

resol = (50,50) # Square image only ! 
print("Resolution of images : ", resol)

In [ ]:
from Retrieving_data.retrieving_data import accessing_2

# These are iterators 
# See fct accessing in codes of Phase 1 
training_set = accessing_2(path_training)
test_set = accessing_2(path_testing) 

In [ ]:
from Retrieving_data.retrieving_data import store_2

# These are arrays
# See fct store in codes of Phase 1 
X_train, y_train = store_2(training_set, [],[], resol)
X_test, y_test = store_2(test_set, [], [], resol)

In [ ]:
from Visualisation.Visu import number
from copy import deepcopy

initial_nbr_train = number(nbr_class, deepcopy(y_train))
initial_nbr_test = number(nbr_class, deepcopy(y_test))

In [ ]:
from Visualisation.Visu import graphs
from copy import deepcopy

graphs(nbr_class, deepcopy(y_train), deepcopy(y_test), 'y_train', 'y_test')

In [ ]:
from keras import Sequential
from keras.layers import RandomZoom, RandomRotation, RandomCrop, RandomContrast
from tensorflow import get_logger
from copy import deepcopy
from numpy import array

# This cell is about data augmentation 
# We'll randomize our set and do our augmentation with this (cells later)
# We choose to work with the second method of augmentation  

#-----------------------------------------------------------------------------

get_logger().setLevel('ERROR')

data_augmentation = Sequential() 

data_augmentation.add(RandomZoom(height_factor=(-0.3, -0.2),width_factor=(-0.3, -0.2)))
data_augmentation.add(RandomRotation((-0.05, 0.05)))
data_augmentation.add(RandomCrop(height=resol[0],width=resol[1]))
data_augmentation.add(RandomContrast((1.0,2.0)))

#-----------------------------------------------------------------------------

# We apply the augmentation on our datasets 
augmented_image_train = array(data_augmentation(deepcopy(X_train)))

#-----------------------------------------------------------------------------

print("Shape of the randomized training set : ", augmented_image_train.shape)

In [ ]:
from scipy.ndimage import gaussian_filter
augmented_image_train = gaussian_filter(augmented_image_train, 0.3)

In [ ]:
from random import randint 
from matplotlib.pyplot import figure, imshow
from numpy import zeros 

# A little test to see the result of the augmentation 

#-----------------------------------------------------------------------------

index = randint(0,len(X_train))

visu_augmented = zeros((50,50,3))

for i in range (3) : 
    for j in range (resol[0]) : 
        for k in range(resol[1]) : 
            visu_augmented [j,k,i] = augmented_image_train[index][j,k,i] / 255

figure()
imshow(X_train[index])
figure()
imshow(visu_augmented)

In [ ]:
from augmentation.aug import first 
from copy import deepcopy

X_train_first, y_train_first = first(nbr_class, deepcopy(initial_nbr_train), deepcopy(initial_nbr_test), deepcopy(augmented_image_train), deepcopy(y_train), resol)

In [ ]:
from Visualisation.Visu import graphs, number

nbr_train_first = number(nbr_class, y_train_first)
graphs(nbr_class, y_train_first.copy(), y_test.copy(), 'y_train_first', 'y_test')

In [ ]:
from augmentation.aug import second 
from copy import deepcopy

X_train_second, y_train_second = second(nbr_class, deepcopy(initial_nbr_train), deepcopy(augmented_image_train), deepcopy(y_train), resol, max(initial_nbr_train))

In [ ]:
from Visualisation.Visu import graphs, number

nbr_train_second = number(nbr_class, y_train_second)
graphs(nbr_class, y_train_second.copy(), y_test.copy(), 'y_train_second', 'y_test')

In [ ]:
from keras.utils import to_categorical
from copy import deepcopy

# We put our results to categorical 

#-----------------------------------------------------------------------------

y_test_tc = to_categorical(deepcopy(y_test), nbr_class)
y_train_tc = to_categorical(deepcopy(y_train), nbr_class)

print("Shape of y_train without augmentation : ", y_train_tc.shape)
print("Shape of y_test without augmentation : ", y_test_tc.shape)

#-----------------------------------------------------------------------------

y_train_first_tc = to_categorical(y_train_first, nbr_class)


print("Shape of y_train with first method : ", y_train_first_tc.shape)
print("Shape of y_test with first method : ", y_test_tc.shape)

#-----------------------------------------------------------------------------

y_train_second_tc = to_categorical(y_train_second, nbr_class)

print("Shape of y_train with second method : ", y_train_second_tc.shape)
print("Shape of y_test with second method : ", y_test_tc.shape)

In [ ]:
import keras_tuner as kt 
from keras.models import Sequential
from keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout

def model_builder(hp):

  model = Sequential()

  # Tune the number of filters for the second Conv2D 
  # Choose an optimal value from 64-128

  hp_k1 = hp.Int('kernel_1', min_value = 2, max_value = 7, step = 1)
  hp_f1 = hp.Int('filter_1', min_value = 64, max_value = 128, step = 4)
  model.add(Conv2D(kernel_size=(hp_k1,hp_k1),filters=hp_f1, activation='relu', input_shape=X_train_second.shape[1:]))

  hp_k2 = hp.Int('kernel_2', min_value = 2, max_value = 5, step = 1) 
  model.add(Conv2D(kernel_size=(hp_k2,hp_k2),filters=hp_f1, activation='relu'))

  hp_p1 = hp.Int('pool_1', min_value = 2, max_value = 5, step = 1)
  model.add(MaxPool2D(pool_size=(hp_p1,hp_p1)))

  hp_d1 = hp.Float("dropout_1", min_value=0.1, max_value=0.5, default=0.25, step=0.05)
  model.add(Dropout(rate=hp_d1))


  hp_k3 = hp.Int('kernel_3', min_value = 2, max_value = 4, step = 1)
  hp_f2 = hp.Int('filter_2', min_value = 128, max_value = 256, step = 16)
  model.add(Conv2D(kernel_size=(hp_k3,hp_k3),filters=hp_f2, activation='relu'))

  hp_k4 = hp.Int('kernel_4', min_value = 2, max_value = 3, step = 1) 
  model.add(Conv2D(kernel_size=(hp_k4,hp_k4),filters=hp_f2, activation='relu'))

  hp_p2 = hp.Int('pool_2', min_value = 2, max_value = 3, step = 1)
  model.add(MaxPool2D(pool_size=(hp_p2,hp_p2)))
  
  hp_d2 = hp.Float("dropout_2", min_value=0.1, max_value=0.5, default=0.25, step=0.05)
  model.add(Dropout(rate=hp_d2))
  

  model.add(Flatten())

  hp_units = hp.Int('dense', min_value = 62, max_value = 302, step = 20)
  model.add(Dense(hp_units, activation = 'relu'))
  model.add(Dense(nbr_class, activation = 'softmax'))

  model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

  return model

In [ ]:
from keras.callbacks import EarlyStopping 

stop_early = EarlyStopping(baseline=0.98)

In [ ]:
tuner = kt.Hyperband(model_builder,
                     objective = 'val_accuracy', 
                     max_epochs = 2,
                     factor = 2,
                     directory = 'my_dir',
                     project_name = 'hyper_tuning',
                     overwrite = True)

In [ ]:
tuner.search(X_train_second,y_train_second_tc, epochs = 2, verbose = 1, validation_data = (X_test, y_test_tc), callbacks = [stop_early])

In [ ]:
# Get the optimal hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials = 1)[0]

print(f"""
The hyperparameter search is complete.\n 
The optimal kernel in first Convolutional layer is  {best_hps.get('kernel_1')}.\n  
The optimal filter in first Convolutional layer is  {best_hps.get('filter_1')}.\n  
The optimal kernel in second Convolutional layer is  {best_hps.get('kernel_2')}.\n
The optimal pool for first pooling layer is {best_hps.get('pool_1')}.\n
The optimal rate for first dropout is {best_hps.get('dropout_1')}.\n
The optimal kernel in third Convolutional layer is  {best_hps.get('kernel_3')}.\n
The optimal filter in second Convolutional layer is  {best_hps.get('filter_2')}.\n
The optimal kernel in fourth Convolutional layer is  {best_hps.get('kernel_4')}.\n
The optimal pool for second pooling layer is {best_hps.get('pool_2')}.\n
The optimal rate for second dropout is {best_hps.get('dropout_2')}.\n
The optimal number of units in the first densely-connected layer is {best_hps.get('dense')}.\n .
""")